In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [20]:
out='../output/'
inb='../input/flows/'
pref='fpw'
inp = inb + pref + '/'

In [21]:
dat=pd.DataFrame(columns=['date','flow','from','hour','to'])
onlyfiles = [f for f in listdir(inp ) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat['flow']=dat['flow'].astype(int)
dat

,date,flow,from,hour,to
0,2016-04-04,4,BOT,0,ZIMA
1,2016-04-04,16,MOZN_EDM,0,ZIMA
2,2016-04-04,16,MOZN_HCB,0,MOZN_EDM
3,2016-04-04,4,RSAN,0,BOT
4,2016-04-04,18,ZAMZ,0,ZAMC
...,...,...,...,...,...
2035,2021-09-30,20,MOZS,23,RSAN
2036,2021-09-30,20,RSAN,23,BOT
2037,2021-09-30,20,ZAMZ,23,ZAMC
2038,2021-09-30,20,ZIM,23,ZAMZ


In [22]:
dam = pd.read_csv(out + pref + '_hourly.csv')
dam=dam.set_index([pd.DatetimeIndex(dam['date']),'hour'])
dam=dam.drop(['date'],1)
dam

price  volume  year  month  day  day_of_week   value  \
date       hour                                                          
2017-03-06 4      29.00    50.0  2017      3    6            0  1450.0   
           5      29.00    50.0  2017      3    6            0  1450.0   
           6      65.00     0.0  2017      3    6            0     0.0   
           7      97.21     0.0  2017      3    6            0     0.0   
           8      97.21     0.0  2017      3    6            0     0.0   
...                 ...     ...   ...    ...  ...          ...     ...   
2021-09-30 19    170.00     0.0  2021      9   30            3     0.0   
           20    107.00     0.0  2021      9   30            3     0.0   
           21    107.00     0.0  2021      9   30            3     0.0   
           22     41.05    40.0  2021      9   30            3  1642.0   
           23     41.05    40.0  2021      9   30            3  1642.0   

                      cat  
date       hour            
2017-03-06 4          off  
           5     standard  
           6      morning  
           7      morning  
           8      morning  
...                   ...  
2021-09-30 19     evening  
           20         off  
           21         off  
           22         off  
           23         off  

[37232 rows x 8 columns]

In [23]:
flow=dat
#flow['date']=pd.to_datetime(flow['flowdate'],format="%d/%m/%Y") #"%Y-%m-%d")#"%d/%m/%Y")
#flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index([pd.DatetimeIndex(flow['date'])])
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek

flow=flow.set_index([pd.DatetimeIndex(flow['date']),'hour'])
flow['price']=dam['price']
flow['value']=flow['price']*flow['flow']
flow=flow.drop(['date'],axis=1)
flow=flow.dropna(0)
flow=flow.reset_index()

flow

,date,hour,flow,from,to,year,month,day,day_of_week,price,value
0,2017-03-06,4,20,BOT,ZIMA,2017,3,6,0,29.00,580.0
1,2017-03-06,4,20,RSAN,BOT,2017,3,6,0,29.00,580.0
2,2017-03-06,4,30,ZAMZ,ZAMC,2017,3,6,0,29.00,870.0
3,2017-03-06,4,20,ZIM,ZAMZ,2017,3,6,0,29.00,580.0
4,2017-03-06,4,20,ZIMA,ZIM,2017,3,6,0,29.00,580.0
...,...,...,...,...,...,...,...,...,...,...,...
117869,2021-09-30,23,20,MOZS,RSAN,2021,9,30,3,41.05,821.0
117870,2021-09-30,23,20,RSAN,BOT,2021,9,30,3,41.05,821.0
117871,2021-09-30,23,20,ZAMZ,ZAMC,2021,9,30,3,41.05,821.0
117872,2021-09-30,23,20,ZIM,ZAMZ,2021,9,30,3,41.05,821.0


In [24]:
f_to=flow[['date','hour','flow','value','price','to','year','month','day','day_of_week']].rename(columns={'to': 'node'})
f_from=flow[['date','hour','flow','value','price','from','year','month','day','day_of_week']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']
f_from['value']=-f_from['value']

flows=pd.concat([f_to, f_from])

flows

,date,hour,flow,value,price,node,year,month,day,day_of_week
0,2017-03-06,4,20,580.0,29.00,ZIMA,2017,3,6,0
1,2017-03-06,4,20,580.0,29.00,BOT,2017,3,6,0
2,2017-03-06,4,30,870.0,29.00,ZAMC,2017,3,6,0
3,2017-03-06,4,20,580.0,29.00,ZAMZ,2017,3,6,0
4,2017-03-06,4,20,580.0,29.00,ZIM,2017,3,6,0
...,...,...,...,...,...,...,...,...,...,...
117869,2021-09-30,23,-20,-821.0,41.05,MOZS,2021,9,30,3
117870,2021-09-30,23,-20,-821.0,41.05,RSAN,2021,9,30,3
117871,2021-09-30,23,-20,-821.0,41.05,ZAMZ,2021,9,30,3
117872,2021-09-30,23,-20,-821.0,41.05,ZIM,2021,9,30,3


In [25]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='HCB'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [26]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows=flows.set_index(['date','hour'])
flows['cat']=dam['cat']
flows.reset_index()
flows

node  year  month  day  day_of_week   index  flow  value  \
date       hour                                                             
2017-03-06 4      RSA  2017      3    6            0       1   -20 -580.0   
           4     ZAMC  2017      3    6            0       2    30  870.0   
           4     ZAMZ  2017      3    6            0       5   -10 -290.0   
           5      RSA  2017      3    6            0       6   -20 -580.0   
           5     ZAMC  2017      3    6            0       7    30  870.0   
...               ...   ...    ...  ...          ...     ...   ...    ...   
2021-09-30 5     ZAMC  2021      9   30            3  117859    20  821.0   
           22     MOZ  2021      9   30            3  117863   -20 -821.0   
           22    ZAMC  2021      9   30            3  117865    20  821.0   
           23     MOZ  2021      9   30            3  117869   -20 -821.0   
           23    ZAMC  2021      9   30            3  117871    20  821.0   

                 price       cat  
date       hour                   
2017-03-06 4     29.00       off  
           4     29.00       off  
           4     58.00       off  
           5     29.00  standard  
           5     29.00  standard  
...                ...       ...  
2021-09-30 5     41.05  standard  
           22    41.05       off  
           22    41.05       off  
           23    41.05       off  
           23    41.05       off  

[48868 rows x 10 columns]

In [27]:
flowsb=flows.copy()
flowsb.loc[flowsb['flow']<0,'side']='supply'
flowsb.loc[flowsb['flow']>0,'side']='consume'


flow_monthly_cat=flowsb.groupby(['year','month', 'node','cat','side']).sum()
flow_monthly_cat.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly_cat['hour_count']=flowsb.groupby(['year','month', 'node','cat','side']).count()['flow']
flow_monthly_cat['price_weight_mean'] = flow_monthly_cat['value'] / flow_monthly_cat['flow_sum']
flow_monthly_cat['price_mean']=flowsb.groupby(['year','month', 'node','cat','side']).mean()['price']
flow_monthly_cat=flow_monthly_cat.drop(['day','day_of_week','index','price'],axis=1)
flow_monthly_cat=flow_monthly_cat.loc[flow_monthly_cat['flow_sum']!=0]
flow_monthly_cat['power']=flow_monthly_cat['flow_sum']/flow_monthly_cat['hour_count']
flow_monthly_cat

flow_sum     value  hour_count  \
year month node cat      side                                      
2017 3     RSA  evening  supply       -200  -17224.0          10   
                morning  consume        50    1748.0           5   
                         supply       -200  -17224.0          10   
                off      consume      1500   36256.0          45   
                         supply      -2880  -83778.4         144   
...                                    ...       ...         ...   
2021 8     ZAMC standard consume       440   24402.6          22   
     9     MOZ  off      supply      -5800 -262095.2         290   
                standard supply       -440  -19662.4          22   
           ZAMC off      consume      5800  262095.2         290   
                standard consume       440   19662.4          22   

                                  price_weight_mean  price_mean      power  
year month node cat      side                                               
2017 3     RSA  evening  supply           86.120000   86.120000 -20.000000  
                morning  consume          34.960000   34.960000  10.000000  
                         supply           86.120000   86.120000 -20.000000  
                off      consume          24.170667   25.968889  33.333333  
                         supply           29.089722   29.089722 -20.000000  
...                                             ...         ...        ...  
2021 8     ZAMC standard consume          55.460455   55.460455  20.000000  
     9     MOZ  off      supply           45.188828   45.188828 -20.000000  
                standard supply           44.687273   44.687273 -20.000000  
           ZAMC off      consume          45.188828   45.188828  20.000000  
                standard consume          44.687273   44.687273  20.000000  

[568 rows x 6 columns]

In [28]:
flow_monthly_cat.to_csv(out + pref + '_flow_monthly_cat.csv')